In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, Dense, Flatten, Dropout, BatchNormalization, Reshape, LeakyReLU
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow as tf
Input = tf.keras.Input

In [3]:
mnist = tf.keras.datasets.mnist

In [4]:
(x_train, y_train), (x_valid, y_valid) = mnist.load_data()
x_train.shape, y_train.shape

((60000, 28, 28), (60000,))

In [5]:
x_train = x_train.reshape(-1, 28, 28, 1)

In [6]:
x_train = x_train / 127.5 - 1
x_train.min(), x_train.max()

(-1.0, 1.0)

In [7]:
encoder_input = Input(shape=(28, 28, 1))

# 28 X 28
x = Conv2D(32, 3, padding='same')(encoder_input) 
x = BatchNormalization()(x)
x = LeakyReLU()(x) 

# 28 X 28 -> 14 X 14
x = Conv2D(64, 3, strides=2, padding='same')(x)
x = BatchNormalization()(x) 
x = LeakyReLU()(x) 

# 14 X 14 -> 7 X 7
x = Conv2D(64, 3, strides=2, padding='same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

# 17 X 7
x = Conv2D(64, 3, padding='same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = Flatten()(x)

# 2D 좌표로 표기하기 위하여 2를 출력값으로 지정합니다.
encoder_output = Dense(2)(x)


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [8]:
encoder = Model(encoder_input, encoder_output)
encoder.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 28, 28, 32)        320       
_________________________________________________________________
batch_normalization (BatchNo (None, 28, 28, 32)        128       
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 28, 28, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
batch_normalization_1 (Batch (None, 14, 14, 64)        256       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 14, 14, 64)        0     

In [9]:
# Input으로는 2D 좌표가 들어갑니다.
decoder_input = Input(shape=(2, ))

# 2D 좌표를 7*7*64 개의 neuron 출력 값을 가지도록 변경합니다.
x = Dense(7*7*64)(decoder_input)
x = Reshape( (7, 7, 64))(x)

# 7 X 7 -> 7 X 7
x = Conv2DTranspose(64, 3, strides=1, padding='same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

# 7 X 7 -> 14 X 14
x = Conv2DTranspose(64, 3, strides=2, padding='same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

# 14 X 14 -> 28 X 28
x = Conv2DTranspose(64, 3, strides=2, padding='same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

# 28 X 28 -> 28 X 28
x = Conv2DTranspose(32, 3, strides=1, padding='same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

# 최종 output
decoder_output = Conv2DTranspose(1, 3, strides=1, padding='same', activation='tanh')(x)

In [10]:
decoder = Model(decoder_input, decoder_output)
decoder.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 2)]               0         
_________________________________________________________________
dense_1 (Dense)              (None, 3136)              9408      
_________________________________________________________________
reshape (Reshape)            (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 7, 7, 64)          36928     
_________________________________________________________________
batch_normalization_4 (Batch (None, 7, 7, 64)          256       
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 14, 14, 64)        3692

In [11]:
LEARNING_RATE = 0.0005
BATCH_SIZE = 32

encoder_in = Input(shape=(28, 28, 1))
x = encoder(encoder_in)
decoder_out = decoder(x)

In [12]:
auto_encoder = Model(encoder_in, decoder_out)

In [13]:
checkpoint_path = 'tmp/01-basic-auto-encoder-MNIST.ckpt'
checkpoint = ModelCheckpoint(checkpoint_path, 
                             save_best_only=True, 
                             save_weights_only=True, 
                             monitor='loss', 
                             verbose=1)

In [14]:
auto_encoder.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
#history = model.fit(X_train, np.array(y_train), epochs=4, batch_size=64, validation_split=0.2)

In [15]:
auto_encoder.fit(x_train, x_train, 
                 batch_size=BATCH_SIZE, 
                 epochs=100, 
                 callbacks=[checkpoint], 
                )
auto_encoder.load_weights(checkpoint_path)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/100
59968/60000 [============================>.] - ETA: 0s - loss: -11.4521 - acc: 3.8028e-04
Epoch 00001: loss improved from inf to -11.45197, saving model to tmp/01-basic-auto-encoder-MNIST.ckpt
60000/60000 [==============================] - 246s 4ms/sample - loss: -11.4520 - acc: 3.8095e-04
Epoch 2/100
59968/60000 [============================>.] - ETA: 0s - loss: -11.6124 - acc: 9.5461e-04
Epoch 00002: loss improved from -11.45197 to -11.61233, saving model to tmp/01-basic-auto-encoder-MNIST.ckpt
60000/60000 [==============================] - 240s 4ms/sample - loss: -11.6123 - acc: 9.5419e-04
Epoch 3/100
59968/60000 [============================>.] - ETA: 0s - loss: -11.6595 - acc: 0.0010
Epoch 00003: loss improved from -11.61233 to -11.65940, saving model to tmp/01-basic-auto-encoder-MNIST.ckpt
60000/60000 [==============================] - 232s 4ms/sample - loss: -11.6594 - acc: 

Epoch 56/100
59968/60000 [============================>.] - ETA: 0s - loss: -11.8551 - acc: 0.0015
Epoch 00056: loss improved from -11.85348 to -11.85523, saving model to tmp/01-basic-auto-encoder-MNIST.ckpt
60000/60000 [==============================] - 235s 4ms/sample - loss: -11.8552 - acc: 0.0015
Epoch 57/100
59968/60000 [============================>.] - ETA: 0s - loss: -11.8595 - acc: 0.0015
Epoch 00057: loss improved from -11.85523 to -11.85948, saving model to tmp/01-basic-auto-encoder-MNIST.ckpt
60000/60000 [==============================] - 234s 4ms/sample - loss: -11.8595 - acc: 0.0015
Epoch 58/100
59968/60000 [============================>.] - ETA: 0s - loss: -11.8610 - acc: 0.0017
Epoch 00058: loss improved from -11.85948 to -11.86102, saving model to tmp/01-basic-auto-encoder-MNIST.ckpt
60000/60000 [==============================] - 235s 4ms/sample - loss: -11.8610 - acc: 0.0017
Epoch 59/100
59968/60000 [============================>.] - ETA: 0s - loss: -11.8620 - acc: 0.